In [26]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.base import TransformerMixin
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

from prettytable import PrettyTable

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.cluster import KMeans

from sklearn.preprocessing import MinMaxScaler
from wordcloud import WordCloud, STOPWORDS

In [27]:
tableData = PrettyTable()
tableData.field_names = ["Model", "Mse"]
print(tableData)

+-------+-----+
| Model | Mse |
+-------+-----+
+-------+-----+


In [2]:
df = pd.read_csv('./cleaned_features.csv')
df.shape

(16664, 16)

## Text Preprocessing: Stemming, stop-word removal and Lemmatization.

Hence in the Preprocessing phase we do the following in the order below:-

1. Begin by removing the html tags
2. Remove any punctuations or limited set of special characters like , or . or # etc.
3. Check if the word is made up of english letters and is not alpha-numeric
4. Check to see if the length of the word is greater than 2 (as it was researched that there is no adjective in 2-letters)
5. Convert the word to lowercase
6. Remove Stopwords
7. Finally Snowball Stemming the word (it was obsereved to be better than Porter Stemming

###  

In [3]:
import re
i=0;
for sent in df['description'].values:
    if (len(re.findall('<.*?>', sent))):
        print(i)
        print(sent)
        break;
    i += 1;

10204
<br />A lovely share apartment in Southfields! You will adore it! Our flat is perfect for solo adventurers, young professionals and students! n- Brightness and comfort n- Excellent location and connection n- Just refurbished n- 5 minutes walking distance from Earsfields Station<br /> We are happy to introduce you to our flat in the middle of Southfields. Our apartment has all you need to enjoy a great stay in the heart of Southwest of London. The place has five fully furnished bedroom and separate toilet and bathroom.  The kitchen is fully equipped and there is a lovely cosy dining area where you can enjoy a meal with your flat mates. You will love cooking here! Our new refurbished apartment has all you need to enjoy a great stay in London. We have decorated this flat with all our creativity, with colourful decorations and a touch of Scandinavian style. - Guests have unlimited access to all amenities and equipment in the flat.  - Please make yourself at home but treat the apartme

In [4]:
nltk.download('stopwords')
stop = set(stopwords.words('english')) #set of stopwords
sno = nltk.stem.SnowballStemmer('english') #initialising the snowball stemmer

def cleanhtml(sentence): #function to clean the word of any html-tags
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', sentence)
    return cleantext
def cleanpunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return  cleaned
print(stop)
print('************************************')
print(sno.stem('tasty'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/raviranjan0631/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
{'they', "isn't", "you'd", 'ma', 'more', "hasn't", 've', 'in', 'his', 'is', 'were', 'other', 'a', "shan't", 'wasn', 'd', 'are', 'itself', 'theirs', 'do', 'off', 'she', 'being', 'both', "hadn't", 'myself', 'what', 'but', 'has', 'be', 'ourselves', 'until', 'on', 'than', 'doing', 'which', 'during', 'no', 'after', 'am', 'couldn', 'all', "you've", 'm', "mustn't", 'same', 'hers', 'them', 'an', 'mightn', "wouldn't", 'those', 'him', 'by', "shouldn't", 'shouldn', 'who', 'as', 'each', 'between', 'it', 'themselves', 'again', 'have', 'their', 'did', 'own', 'mustn', 'that', 'our', 'the', "you'll", 'your', 'haven', 'had', 'any', "haven't", 'further', 's', 'll', 'how', 'few', 'will', "aren't", 'weren', 'won', 'some', 'whom', 'once', 'too', 'we', 'under', 't', 'yourselves', 'down', 'himself', 'herself', 'yourself', 'at', 'up', 'isn', 'here', 'not', "shou

In [5]:
# this code takes a while to run as it needs to run on 10k sentences.
i=0
str1=' '
final_string=[]
s=''
for sent in df['description'].values:
    filtered_sentence=[]
    #print(sent);
    sent=cleanhtml(sent) # remove HTMl tags
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if((cleaned_words.isalpha()) & (len(cleaned_words)>2)):    
                if(cleaned_words.lower() not in stop):
                    s=(sno.stem(cleaned_words.lower())).encode('utf8')
                    filtered_sentence.append(s)
                else:
                    continue
            else:
                continue 
    #print(filtered_sentence)
    str1 = b" ".join(filtered_sentence) #final string of cleaned words
    #print("***********************************************************************")
    
    final_string.append(str1)
    i+=1

In [6]:
df['CleanedText']=final_string #adding a column of CleanedText which displays the data after pre-processing of the review 
df['CleanedText']=df['CleanedText'].str.decode("utf-8")
df.head()

,description,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,price,cleaning_fee,guests_included,extra_people,minimum_nights,reviews_per_month,CleanedText
0,Lots of windows and light. St Luke's Gardens ...,51.48796,-0.16898,Apartment,Entire home/apt,2,1.0,1.0,1.0,Real Bed,100.0,50.0,2,0.0,3,0.73,lot window light luke garden end block river f...
1,"Open from June 2018 after a 3-year break, we a...",51.52098,-0.14002,Apartment,Entire home/apt,6,2.0,3.0,3.0,Real Bed,300.0,65.0,4,10.0,3,0.41,open june break delight welcom guest superb mo...
2,Big room with double bed/ clean sheets/ clean ...,51.57224,-0.20906,Others,Private room,2,1.5,1.0,1.0,Real Bed,29.0,0.0,1,8.0,10,1.08,big room doubl bed clean sheet clean towel cle...
3,Situated literally in London Fields park and o...,51.53972,-0.05885,Apartment,Entire home/apt,2,1.0,1.0,1.0,Real Bed,80.0,30.0,1,0.0,6,0.48,situat liter london field park moment lido hea...
4,Private lockable bedsit room available within ...,51.52589,-0.19942,Apartment,Private room,2,1.0,1.0,1.0,Not Real bed,29.0,15.0,1,10.0,5,0.13,privat lockabl bedsit room avail within bright...


## TF-IDF operation

In [7]:
## TF-IDF
tf_idf = TfidfVectorizer(ngram_range=(1,2),min_df = 10)
tfidf = tf_idf.fit_transform(df['CleanedText'].values)
print('The tfidf vectors shape of data is ',tfidf.shape)

The tfidf vectors shape of data is  (16664, 23854)


In [8]:
df.columns

Index(['description', 'latitude', 'longitude', 'property_type', 'room_type',
       'accommodates', 'bathrooms', 'bedrooms', 'beds', 'bed_type', 'price',
       'cleaning_fee', 'guests_included', 'extra_people', 'minimum_nights',
       'reviews_per_month', 'CleanedText'],
      dtype='object')

In [9]:
y = df.price.values
df = df.drop(['description', 'price', 'CleanedText'], axis=1)

In [10]:
# Dummy encoding
categorical_feats = ['property_type', 'room_type', 'bed_type']
df = pd.get_dummies(df, columns=categorical_feats, drop_first=False)
print("Dataset has {} rows, {} columns.".format(*df.shape))

Dataset has 16664 rows, 18 columns.


In [ ]:
# column standarization
df['accommodates'] = (df['accommodates'] - df['accommodates'].mean()) / df['accommodates'].std()
df['bathrooms'] = (df['bathrooms'] - df['bathrooms'].mean()) / df['bathrooms'].std()
df['bedrooms'] = (df['bedrooms'] - df['bedrooms'].mean()) / df['bedrooms'].std()
df['cleaning_fee'] = (df['cleaning_fee'] - df['cleaning_fee'].mean()) / df['cleaning_fee'].std()
df['minimum_nights'] = (df['minimum_nights'] - df['minimum_nights'].mean()) / df['minimum_nights'].std()
df['extra_people'] = (df['extra_people'] - df['extra_people'].mean()) / df['extra_people'].std()
df['beds'] = (df['beds'] - df['beds'].mean()) / df['beds'].std()
df['guests_included'] = (df['guests_included'] - df['guests_included'].mean()) / df['guests_included'].std()
df['reviews_per_month'] = (df['reviews_per_month'] - df['reviews_per_month'].mean()) / df['reviews_per_month'].std()


df['tfidf'] = tfidf

In [14]:
df.shape

(16664, 18)

In [ ]:
df.columns

In [17]:
X = df.values

In [18]:
test_size = 0.3
seed = 42

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

In [19]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

## Linear Regression

In [20]:
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [28]:
# Make predictions using the testing set
regr_y_pred = regr.predict(X_test)

# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(y_test, regr_y_pred))

Mean squared error: 6314.74


In [30]:
 tableData.add_row(["Linear Regression", mean_squared_error(y_test, regr_y_pred)])

## Random Forest

In [23]:
#Tuning the model
param_grid = { "n_estimators"      : [175, 200, 225, 250],
           "criterion"         : ['mse'],
           "max_features"      : ['auto'], #auto, sqrt, log2, int/n_feature
           "max_depth"         : [10, 14, 15, 16, 18],
           "min_samples_split" : [6, 7, 8, 9, 10] ,
           "bootstrap": [True]}


In [24]:
%%time
rf = RandomForestRegressor(random_state=seed)

rf_cv = GridSearchCV(rf, param_grid, cv=5)
rf_cv.fit(X_train, y_train)

CPU times: user 1h 33min 42s, sys: 10 s, total: 1h 33min 52s
Wall time: 1h 34min 6s


In [25]:
# Make prediction
rf_y_pred = rf_cv.predict(X_test)

# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(y_test, rf_y_pred))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y_test, rf_y_pred))
# Best params
print("Tuned Model Parameters: {}".format(rf_cv.best_params_))

Mean squared error: 3982.48
Variance score: 0.64
Tuned Model Parameters: {'bootstrap': True, 'criterion': 'mse', 'max_depth': 18, 'max_features': 'auto', 'min_samples_split': 6, 'n_estimators': 250}


In [31]:
 tableData.add_row(["Random Forest Regression", mean_squared_error(y_test, rf_y_pred)])

In [33]:
print(tableData)

+--------------------------+-------------------+
|          Model           |        Mse        |
+--------------------------+-------------------+
|    Linear Regression     | 6314.743795004338 |
| Random Forest Regression |  3982.47511449271 |
+--------------------------+-------------------+


### Observation 

We get minimum Mse in Random Forest